In [1]:
import torch 
import torch.nn as nn
import torchaudio
import numpy as np
from datasets import load_dataset, load_metric, Audio
import os, glob
import json
sr = 16000

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
torch.backends.cudnn.benchmark = True

In [3]:
tgt = 'roman'

In [4]:
import pandas as pd
val = pd.read_csv(f'./val_{tgt}.csv', index_col=0)
val = val.reset_index()
val.head()

,index,filepath,char,hira,phone,roman
0,0,./data/wav/BASIC5000_0001.wav,水 を マ レ ー シ ア か ら 買 わ な く て は な ら な い の で す,み ず を ま れ ー し あ か ら か わ な く て わ な ら な い の で す,m i z u o m a r e e s h ...,mizuwomareeshiakarakawanakutewanaranainodesu
1,1,./data/wav/BASIC5000_0002.wav,木 曜 日 停 戦 会 談 は 何 の 進 展 も な い ま ま 終 了 し ま し た,も く よ ー び て ー せ ん か い だ ん わ な ん の し ん て ん も な ...,m o k u y o o b i p a u t ...,mokuyoobiteesenkaidanwanannoshintenmonaimamash...
2,2,./data/wav/BASIC5000_0003.wav,上 院 議 員 は 私 が デ ー タ を ゆ が め た と 告 発 し た,じ ょ ー い ん ぎ ー ん わ わ た し が で ー た を ゆ が め た と こ ...,j o o i N g i i N w a p ...,jooingiinwawatashigadeetawoyugametatokokuhatsu...
3,3,./data/wav/BASIC5000_0004.wav,１ 週 間 し て そ の ニ ュ ー ス は 本 当 に な っ た,い っ し ゅ ー か ん し て そ の に ゅ ー す わ ほ ん と ー に な っ た,i c l s h u u k a N s h i ...,isshuukanshitesononyuusuwahontooninatta
4,4,./data/wav/BASIC5000_0005.wav,血 圧 は 健 康 の パ ロ メ ー タ ー と し て 重 要 で あ る,け つ あ つ わ け ん こ ー の ぱ ろ め ー た ー と し て じ ゅ ー よ ...,k e t s u a t s u w a p a u ...,ketsuatsuwakenkoonoparomeetaatoshitejuuyoodearu


In [5]:
model_folder = f'wav2vec2_tiny_ja_{tgt}'

In [6]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(f"./vocab_{tgt}.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
tokenizer

PreTrainedTokenizer(name_or_path='', vocab_size=2699, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [7]:
token_list = open(f"./vocab_{tgt}.json",'r')
token_list = json.load(token_list)
token_list = {k: v for k, v in enumerate(token_list)}
token_list 

{0: '-',
 1: '0',
 2: '1',
 3: '5',
 4: 'C',
 5: 'P',
 6: '々',
 7: 'ぁ',
 8: 'あ',
 9: 'い',
 10: 'う',
 11: 'え',
 12: 'ぉ',
 13: 'お',
 14: 'か',
 15: 'が',
 16: 'き',
 17: 'ぎ',
 18: 'く',
 19: 'ぐ',
 20: 'け',
 21: 'げ',
 22: 'こ',
 23: 'ご',
 24: 'さ',
 25: 'ざ',
 26: 'し',
 27: 'じ',
 28: 'す',
 29: 'ず',
 30: 'せ',
 31: 'ぜ',
 32: 'そ',
 33: 'ぞ',
 34: 'た',
 35: 'だ',
 36: 'ち',
 37: 'っ',
 38: 'つ',
 39: 'づ',
 40: 'て',
 41: 'で',
 42: 'と',
 43: 'ど',
 44: 'な',
 45: 'に',
 46: 'ぬ',
 47: 'ね',
 48: 'の',
 49: 'は',
 50: 'ば',
 51: 'ぱ',
 52: 'ひ',
 53: 'び',
 54: 'ぴ',
 55: 'ふ',
 56: 'ぶ',
 57: 'ぷ',
 58: 'へ',
 59: 'べ',
 60: 'ぺ',
 61: 'ほ',
 62: 'ぼ',
 63: 'ぽ',
 64: 'ま',
 65: 'み',
 66: 'む',
 67: 'め',
 68: 'も',
 69: 'ゃ',
 70: 'や',
 71: 'ゅ',
 72: 'ゆ',
 73: 'ょ',
 74: 'よ',
 75: 'ら',
 76: 'り',
 77: 'る',
 78: 'れ',
 79: 'ろ',
 80: 'わ',
 81: 'を',
 82: 'ん',
 83: 'ァ',
 84: 'ア',
 85: 'ィ',
 86: 'イ',
 87: 'ゥ',
 88: 'ウ',
 89: 'ェ',
 90: 'エ',
 91: 'ォ',
 92: 'オ',
 93: 'カ',
 94: 'ガ',
 95: 'キ',
 96: 'ギ',
 97: 'ク',
 98: 'グ',
 99: 'ケ',
 100: 'ゲ',

In [8]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=sr, padding_value=0.0, do_normalize=True, return_attention_mask=True)
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [9]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [10]:
class W2v2Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.pathes = df['filepath'].values
        self.sentences = df[tgt].values

    def __getitem__(self, idx):
        waveform, sample_rate = torchaudio.load(self.pathes[idx])
        batch = dict()
        batch["input_values"] = processor(waveform.reshape(-1), sampling_rate=sr).input_values[0]  
        with processor.as_target_processor():
            batch["labels"] = processor(self.sentences[idx]).input_ids       
        
        return batch

    def __len__(self):
        return len(self.df)

val_dataset = W2v2Dataset(val)

In [11]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [12]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [13]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    'charsiu/zh_w2v2_tiny_fc_10ms',
    attention_dropout=0.2,
    hidden_dropout=0.2,
    feat_proj_dropout=0.2,
    mask_time_prob=0.075,
    layerdrop=0.2,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    diversity_loss_weight=100
    #vocab_size=len(processor.tokenizer)
)

model.lm_head = nn.Linear(384, len(processor.tokenizer))
model.config.vocab_size=len(processor.tokenizer)

In [14]:
model_ckpts = glob.glob(f'{model_folder}/ch*')
model_ckpt = sorted(model_ckpts)[-1]
model_file = f'./{model_ckpt}/pytorch_model.bin'
model.load_state_dict(torch.load(model_file))

<All keys matched successfully>

In [15]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model = model.to(device)

In [16]:
from torch.utils.data import DataLoader
test_loader = DataLoader(val_dataset,
                             batch_size=1,
                             shuffle=False,
                             num_workers=4)

In [17]:
# 認識結果が記述されたファイル
hypothesis_file = os.path.join(f'./hypothesis_{tgt}.txt')

# 正解文が記述されたファイル
reference_file = os.path.join(f'./reference_{tgt}.txt')

In [18]:
import torchaudio as ta
with open(hypothesis_file, mode='w') as hyp_file, \
     open(reference_file, mode='w') as ref_file:
    for i in range(len(val)):
        input_values = processor(ta.load(val['filepath'].iloc[i])[0], return_tensors="pt", sampling_rate=sr).input_values.to(device)  # Batch size 1
        logits = model(input_values[0]).logits
        predicted_ids = torch.argmax(logits, dim=-1)

        transcription = processor.decode(predicted_ids[0])
        gt = val.iloc[i][tgt]
        hyp_file.write('%s %s\n' \
                % (i, ' '.join(transcription)))
        ref_file.write('%s %s\n' \
            % (i, ' '.join(gt)))
    

In [19]:
result_file = os.path.join(f'result_{tgt}.txt')

In [20]:
# -*- coding: utf-8 -*-

#
# レーベンシュタイン距離を用いて，
# 認識結果の誤り数を算出します．
#

import numpy as np
import copy

def calculate_error(hypothesis, reference):
    ''' レーベンシュタイン距離を計算し，
        置換誤り，削除誤り，挿入誤りを出力する
    hypothesis:       認識結果(トークン毎に区切ったリスト形式)
    reference:        正解(同上)
    total_error:      総誤り数
    substitute_error: 置換誤り数
    delete_error:     削除誤り数
    insert_error:     挿入誤り数
    len_ref:          正解文のトークン数
    '''
    # 認識結果および正解系列の長さを取得
    len_hyp = len(hypothesis)
    len_ref = len(reference)

    # 累積コスト行列を作成する
    # 行列の各要素には，トータルコスト，
    # 置換コスト，削除コスト，挿入コストの
    # 累積値が辞書形式で定義される．
    cost_matrix = [[{"total":0, 
                     "substitute":0,
                     "delete":0,
                     "insert":0} \
                     for j in range(len_ref+1)] \
                         for i in range(len_hyp+1)]

    # 0列目と0行目の入力
    for i in range(1, len_hyp+1):
        # 縦方向への遷移は，削除処理を意味する
        cost_matrix[i][0]["delete"] = i
        cost_matrix[i][0]["total"] = i
    for j in range(1, len_ref+1):
        # 横方向への遷移は，挿入処理を意味する
        cost_matrix[0][j]["insert"] = j
        cost_matrix[0][j]["total"] = j

    # 1列目と1行目以降の累積コストを計算していく
    for i in range(1, len_hyp+1):
        for j in range(1, len_ref+1):
            #
            # 各処理のコストを計算する
            #
            # 斜め方向の遷移時，文字が一致しない場合は，
            # 置換処理により累積コストが1増加
            substitute_cost = \
                cost_matrix[i-1][j-1]["total"] \
                + (0 if hypothesis[i-1] == reference[j-1] else 1)
            # 縦方向の遷移時は，削除処理により累積コストが1増加
            delete_cost = cost_matrix[i-1][j]["total"] + 1
            # 横方向の遷移時は，挿入処理により累積コストが1増加
            insert_cost = cost_matrix[i][j-1]["total"] + 1

            # 置換処理，削除処理，挿入処理のうち，
            # どの処理を行えば累積コストが最も小さくなるかを計算
            cost = [substitute_cost, delete_cost, insert_cost]
            min_index = np.argmin(cost)

            if min_index == 0:
                # 置換処理が累積コスト最小となる場合

                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = \
                    copy.copy(cost_matrix[i-1][j-1])
                # 文字が一致しない場合は，
                # 累積置換コストを1増加させる
                cost_matrix[i][j]["substitute"] \
                    += (0 if hypothesis[i-1] \
                        == reference[j-1] else 1)
            elif min_index == 1:
                # 削除処理が累積コスト最小となる場合
                
                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = copy.copy(cost_matrix[i-1][j])
                # 累積削除コストを1増加させる
                cost_matrix[i][j]["delete"] += 1
            else:
                # 置換処理が累積コスト最小となる場合
                
                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = copy.copy(cost_matrix[i][j-1])
                # 累積挿入コストを1増加させる
                cost_matrix[i][j]["insert"] += 1

            # 累積トータルコスト(置換+削除+挿入コスト)を更新
            cost_matrix[i][j]["total"] = cost[min_index]

    #
    # エラーの数を出力する
    # このとき，削除コストは挿入誤り，
    # 挿入コストは削除誤りになる点に注意．
    # (削除コストが1である
    #    = 1文字削除しないと正解文にならない 
    #    = 認識結果は1文字分余計に挿入されている
    #    = 挿入誤りが1である)
    #

    # 累積コスト行列の右下の要素が最終的なコストとなる．
    total_error = cost_matrix[len_hyp][len_ref]["total"]
    substitute_error = cost_matrix[len_hyp][len_ref]["substitute"]
    # 削除誤り = 挿入コスト
    delete_error = cost_matrix[len_hyp][len_ref]["insert"]
    # 挿入誤り = 削除コスト
    insert_error = cost_matrix[len_hyp][len_ref]["delete"]
    
    # 各誤り数と，正解文の文字数
    # (誤り率を算出する際に分母として用いる)を出力
    return (total_error, 
            substitute_error,
            delete_error,
            insert_error,
            len_ref)


if __name__ == "__main__":
    # ref: 正解文
    # hyp: 認識結果
    ref = "狼が犬に似ているようにおべっか使いは友達のように見える"
    hyp = "オオ狼みがい塗に似ているようにオッつ界は伴ちのように見える"


    # 各文字列を，1文字ずつ区切ってリストデータにする
    hyp_list = list(hyp)
    ref_list = list(ref)

    # 誤り数を計算する# -*- coding: utf-8 -*-

#
# レーベンシュタイン距離を用いて，
# 認識結果の誤り数を算出します．
#

import numpy as np
import copy

def calculate_error(hypothesis, reference):
    ''' レーベンシュタイン距離を計算し，
        置換誤り，削除誤り，挿入誤りを出力する
    hypothesis:       認識結果(トークン毎に区切ったリスト形式)
    reference:        正解(同上)
    total_error:      総誤り数
    substitute_error: 置換誤り数
    delete_error:     削除誤り数
    insert_error:     挿入誤り数
    len_ref:          正解文のトークン数
    '''
    # 認識結果および正解系列の長さを取得
    len_hyp = len(hypothesis)
    len_ref = len(reference)

    # 累積コスト行列を作成する
    # 行列の各要素には，トータルコスト，
    # 置換コスト，削除コスト，挿入コストの
    # 累積値が辞書形式で定義される．
    cost_matrix = [[{"total":0, 
                     "substitute":0,
                     "delete":0,
                     "insert":0} \
                     for j in range(len_ref+1)] \
                         for i in range(len_hyp+1)]

    # 0列目と0行目の入力
    for i in range(1, len_hyp+1):
        # 縦方向への遷移は，削除処理を意味する
        cost_matrix[i][0]["delete"] = i
        cost_matrix[i][0]["total"] = i
    for j in range(1, len_ref+1):
        # 横方向への遷移は，挿入処理を意味する
        cost_matrix[0][j]["insert"] = j
        cost_matrix[0][j]["total"] = j

    # 1列目と1行目以降の累積コストを計算していく
    for i in range(1, len_hyp+1):
        for j in range(1, len_ref+1):
            #
            # 各処理のコストを計算する
            #
            # 斜め方向の遷移時，文字が一致しない場合は，
            # 置換処理により累積コストが1増加
            substitute_cost = \
                cost_matrix[i-1][j-1]["total"] \
                + (0 if hypothesis[i-1] == reference[j-1] else 1)
            # 縦方向の遷移時は，削除処理により累積コストが1増加
            delete_cost = cost_matrix[i-1][j]["total"] + 1
            # 横方向の遷移時は，挿入処理により累積コストが1増加
            insert_cost = cost_matrix[i][j-1]["total"] + 1

            # 置換処理，削除処理，挿入処理のうち，
            # どの処理を行えば累積コストが最も小さくなるかを計算
            cost = [substitute_cost, delete_cost, insert_cost]
            min_index = np.argmin(cost)

            if min_index == 0:
                # 置換処理が累積コスト最小となる場合

                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = \
                    copy.copy(cost_matrix[i-1][j-1])
                # 文字が一致しない場合は，
                # 累積置換コストを1増加させる
                cost_matrix[i][j]["substitute"] \
                    += (0 if hypothesis[i-1] \
                        == reference[j-1] else 1)
            elif min_index == 1:
                # 削除処理が累積コスト最小となる場合
                
                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = copy.copy(cost_matrix[i-1][j])
                # 累積削除コストを1増加させる
                cost_matrix[i][j]["delete"] += 1
            else:
                # 置換処理が累積コスト最小となる場合
                
                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = copy.copy(cost_matrix[i][j-1])
                # 累積挿入コストを1増加させる
                cost_matrix[i][j]["insert"] += 1

            # 累積トータルコスト(置換+削除+挿入コスト)を更新
            cost_matrix[i][j]["total"] = cost[min_index]

    #
    # エラーの数を出力する
    # このとき，削除コストは挿入誤り，
    # 挿入コストは削除誤りになる点に注意．
    # (削除コストが1である
    #    = 1文字削除しないと正解文にならない 
    #    = 認識結果は1文字分余計に挿入されている
    #    = 挿入誤りが1である)
    #

    # 累積コスト行列の右下の要素が最終的なコストとなる．
    total_error = cost_matrix[len_hyp][len_ref]["total"]
    substitute_error = cost_matrix[len_hyp][len_ref]["substitute"]
    # 削除誤り = 挿入コスト
    delete_error = cost_matrix[len_hyp][len_ref]["insert"]
    # 挿入誤り = 削除コスト
    insert_error = cost_matrix[len_hyp][len_ref]["delete"]
    
    # 各誤り数と，正解文の文字数
    # (誤り率を算出する際に分母として用いる)を出力
    return (total_error, 
            substitute_error,
            delete_error,
            insert_error,
            len_ref)


if __name__ == "__main__":
    # ref: 正解文
    # hyp: 認識結果
    ref = "狼が犬に似ているようにおべっか使いは友達のように見える"
    hyp = "オオ狼みがい塗に似ているようにオッつ界は伴ちのように見える"


    # 各文字列を，1文字ずつ区切ってリストデータにする
    hyp_list = list(hyp)
    ref_list = list(ref)

    # 誤り数を計算する
    total, substitute, delete, insert, ref_length \
        = calculate_error(hyp_list, ref_list)

    # 誤りの数と，誤り率(100*誤り数/正解文の文字数)を出力する
    print("REF: %s" % ref)
    print("HYP: %s" % hyp)
    print("#TOKEN(REF): %d, #ERROR: %d, #SUB: %d, #DEL: %d, #INS: %d" \
        % (ref_length, total, substitute, delete, insert))
    print("UER: %.2f, SUBR: %.2f, DELR: %.2f, INSR: %.2f" \
        % (100.0*total/ref_length,
           100.0*substitute/ref_length,
           100.0*delete/ref_length,
           100.0*insert/ref_length))


    total, substitute, delete, insert, ref_length \
        = calculate_error(hyp_list, ref_list)

    # 誤りの数と，誤り率(100*誤り数/正解文の文字数)を出力する
    print("REF: %s" % ref)
    print("HYP: %s" % hyp)
    print("#TOKEN(REF): %d, #ERROR: %d, #SUB: %d, #DEL: %d, #INS: %d" \
        % (ref_length, total, substitute, delete, insert))
    print("UER: %.2f, SUBR: %.2f, DELR: %.2f, INSR: %.2f" \
        % (100.0*total/ref_length,
           100.0*substitute/ref_length,
           100.0*delete/ref_length,
           100.0*insert/ref_length))



REF: 狼が犬に似ているようにおべっか使いは友達のように見える
HYP: オオ狼みがい塗に似ているようにオッつ界は伴ちのように見える
#TOKEN(REF): 27, #ERROR: 13, #SUB: 7, #DEL: 2, #INS: 4
UER: 48.15, SUBR: 25.93, DELR: 7.41, INSR: 14.81
REF: 狼が犬に似ているようにおべっか使いは友達のように見える
HYP: オオ狼みがい塗に似ているようにオッつ界は伴ちのように見える
#TOKEN(REF): 27, #ERROR: 13, #SUB: 7, #DEL: 2, #INS: 4
UER: 48.15, SUBR: 25.93, DELR: 7.41, INSR: 14.81


In [21]:
# 各誤りの総数(エラー率算出時の分子)
total_err = 0
total_sub = 0
total_del = 0
total_ins = 0
# 正解文の総文字数(エラー率算出時の分母)
total_length = 0

# 各ファイルをオープン
with open(hypothesis_file, mode='r') as hyp_file, \
     open(reference_file, mode='r') as ref_file, \
     open(result_file, mode='w') as out_file:
    # 認識結果ファイル正解文ファイルを一行ずつ読み込む
    for line_hyp, line_ref in zip(hyp_file, ref_file):
        # 読み込んだ行をスペースで区切り，リスト型の変数にする
        parts_hyp = line_hyp.split()
        parts_ref = line_ref.split()

        # 発話ID(partsの0番目の要素)が一致しているか確認
        if parts_hyp[0] != parts_ref[0]:
            sys.stderr.write('Utterance ids of '\
                'hypothesis and reference do not match.')
            exit(1)

        # 1要素目以降が認識結果/正解分の文字列(リスト型)
        hypothesis = parts_hyp[1:]
        reference = parts_ref[1:]

        # 誤り数を計算する
        (error, substitute, delete, insert, ref_length) \
            = calculate_error(hypothesis,
                                          reference)

        # 総誤り数を累積する
        total_err += error
        total_sub += substitute
        total_del += delete
        total_ins += insert
        total_length += ref_length

        # 各発話の結果を出力する
        out_file.write('ID: %s\n' % (parts_hyp[0]))
        out_file.write('#ERROR (#SUB #DEL #INS): '\
            '%d (%d %d %d)\n' \
            % (error, substitute, delete, insert))
        out_file.write('REF: %s\n' % (' '.join(reference)))
        out_file.write('HYP: %s\n' % (' '.join(hypothesis)))
        out_file.write('\n')

    # 総エラー数を，正解文の総文字数で割り，エラー率を算出する
    err_rate = 100.0 * total_err / total_length
    sub_rate = 100.0 * total_sub / total_length
    del_rate = 100.0 * total_del / total_length
    ins_rate = 100.0 * total_ins / total_length

    # 最終結果を出力する
    out_file.write('------------------------------'\
        '-----------------------------------------------\n')
    out_file.write('#TOKEN: %d, #ERROR: %d '\
        '(#SUB: %d, #DEL: %d, #INS: %d)\n' \
        % (total_length, total_err,
           total_sub, total_del, total_ins))
    out_file.write('TER: %.2f%% (SUB: %.2f, '\
        'DEL: %.2f, INS: %.2f)\n' \
        % (err_rate, sub_rate, del_rate, ins_rate))
    print('TER: %.2f%% (SUB: %.2f, DEL: %.2f, INS: %.2f)' \
        % (err_rate, sub_rate, del_rate, ins_rate))
    out_file.write('------------------------------'\
        '-----------------------------------------------\n')



TER: 33.49% (SUB: 22.62, DEL: 9.29, INS: 1.58)
